In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Import Graphs

In [6]:
import os
import networkx as nx
import pickle
# from solvers.KaMIS import ReduMIS

graph_directory = "../../graphs/er_700-800/"

datasets = []

for filename in os.listdir(graph_directory):
    if filename.endswith(".gpickle"):
        print("ER 700-800 Graph ", os.path.join(graph_directory, filename), "is being imported ...")
        with open(os.path.join(graph_directory, filename), 'rb') as f:
            G = pickle.load(f)
            datasets.append(
            {
                "name": filename[:-8],
                "graph": nx.relabel.convert_node_labels_to_integers(
                    G, first_label=0
                ),
            })
            # ReduMIS.networkx_to_metis(G, f"../../test/er/{filename.split('.')[0]}.{filename.split('.')[1]}.metis")

datasets = datasets[0:10]
print(datasets)

ER 700-800 Graph  ../../graphs/er_700-800/ER_700_800_0.15_76.gpickle is being imported ...
ER 700-800 Graph  ../../graphs/er_700-800/ER_700_800_0.15_111.gpickle is being imported ...
ER 700-800 Graph  ../../graphs/er_700-800/ER_700_800_0.15_86.gpickle is being imported ...
ER 700-800 Graph  ../../graphs/er_700-800/ER_700_800_0.15_22.gpickle is being imported ...
ER 700-800 Graph  ../../graphs/er_700-800/ER_700_800_0.15_39.gpickle is being imported ...
ER 700-800 Graph  ../../graphs/er_700-800/ER_700_800_0.15_78.gpickle is being imported ...
ER 700-800 Graph  ../../graphs/er_700-800/ER_700_800_0.15_85.gpickle is being imported ...
ER 700-800 Graph  ../../graphs/er_700-800/ER_700_800_0.15_84.gpickle is being imported ...
ER 700-800 Graph  ../../graphs/er_700-800/ER_700_800_0.15_26.gpickle is being imported ...
ER 700-800 Graph  ../../graphs/er_700-800/ER_700_800_0.15_93.gpickle is being imported ...
ER 700-800 Graph  ../../graphs/er_700-800/ER_700_800_0.15_60.gpickle is being imported ..

## Define Solver Parameters

In [3]:
# from solvers.dNNMIS_GPU_TAU import DNNMIS as V2
from solvers.maxcut.Gurobi import Gurobi

solvers = [

    {
        "name": "Gurobi",
        "class": Gurobi,
        "params": {
            "time_limit": 10,
        },
    },
    # {
    #     "name": "dNN V2 GPU",
    #     "class": V2,
    #     "params": {
    #         "learning_rate": 0.05,
    #         "selection_criteria": 0.45,
    #         "max_steps": 20000
    #     },
    # },
    # {
    #     "name": "Quadratic Standard",
    #     "class": Quadratic,
    #     "params": {
    #         "learning_rate": 0.5,
    #         "number_of_steps": 4000,
    #         "gamma": 625,
    #         "batch_size": 150,
    #         "lr_gamma": 0.2
    #     },
    # },
    # {
    #     "name": "Quadratic Normalized",
    #     "class": Quadratic,
    #     "params": {
    #         "learning_rate": 0.5,
    #         "number_of_steps": 4000,
    #         "gamma": 625,
    #         "batch_size": 150,
    #         "lr_gamma": 0.2,
    #         "normalize": True
    #     },
    # },
    # {
    #     "name": "Quadratic Combined",
    #     "class": Quadratic,
    #     "params": {
    #         "learning_rate": 0.5,
    #         "number_of_steps": 4000,
    #         "gamma": 625,
    #         "batch_size": 150,
    #         "lr_gamma": 0.2,
    #         "combine": True
    #     },
    # },
    # {
    #     "name": "dNN",
    #     "class": DNNMIS,
    #     "params": {
    #         "learning_rate": 0.001,
    #         "selection_criteria": 0.8,
    #         "max_steps": 25000,
    #     },
    # },
    #     {
    #     "name": "dNN w/SG5k",
    #     "class": DNNMIS,
    #     "params": {
    #         "learning_rate": 0.001,
    #         "selection_criteria": 0.65,
    #         "max_steps": 80000,
    #         "max_subgraph_steps": 5000,
    #     },
    # },
    # {"name": "ReduMIS", "class": ReduMIS, "params": {"seed": 13}},
    # {"name": "ILP", "class": ILPMIS, "params": {"time_limit": 300}}
]

## Optional: Run New Benchmark

In [4]:
from copy import deepcopy


def benchmark(datasets, solvers):
    solutions = []

    stage = 0
    stages = len(solvers) * len(datasets)


    for solver in solvers:
        for dataset in datasets:
            solver_instance = solver["class"](dataset["graph"], solver["params"])
            print(solver_instance.solve())
            # solution = {
            #     "solution_method": solver['name'],
            #     "dataset_name": dataset["name"],
            #     "data": deepcopy(solver_instance.solution),
            #     "time_taken": deepcopy(solver_instance.solution_time),
            # }
            # solutions.append(solution)
            del solver_instance
            stage += 1
            print(f"Completed {stage} / {stages}")

    return solutions

solutions = benchmark(datasets, solvers)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2509113
Academic license 2509113 - for non-commercial use only - registered to le___@osu.edu
Set parameter TimeLimit to value 10
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Academic license 2509113 - for non-commercial use only - registered to le___@osu.edu
Optimize a model with 0 rows, 742 columns and 0 nonzeros
Model fingerprint: 0xc9f72c3a
Model has 41136 quadratic objective terms
Variable types: 0 continuous, 742 integer (742 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [8e+01, 1e+02]
  QObjective range [4e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objectiv

## Generate Results Table

In [5]:
import pandas
import matplotlib.pyplot as plt
from itertools import combinations

dataset_index = {k: v for v, k in enumerate([dataset["name"] for dataset in datasets])}
datasets_solutions = [[] for i in range(len(datasets))]

def MIS_checker(MIS_list, G):
    pairs = list(combinations(MIS_list, 2))
    IS_CHECKER = True
    if len(MIS_list) > 1:
        for pair in pairs:
            if (pair[0], pair[1]) in G.edges or (pair[1], pair[0]) in G.edges:
                IS_CHECKER = False
                break
    return IS_CHECKER

table_data = []

for solution in solutions:
    dsi = dataset_index[solution["dataset_name"]]
    datasets_solutions[dsi].append(solution)

i = 0
for dataset_solutions in datasets_solutions:
    # IS CHECK
    is_check = []
    for solution in dataset_solutions:
        IS_set = solution["data"]["graph_mask"]
        # print(IS_set)
        # indices = [i for i, x in enumerate(IS_set) if x == 1]
        # print(indices)
        # subgraph = datasets[dataset_index[solution["dataset_name"]]]["graph"].subgraph(indices)
        # subgraph = nx.Graph(subgraph)
        # while len(subgraph) > 0:
        #     degrees = dict(subgraph.degree())
        #     max_degree_nodes = [node for node, degree in degrees.items() if degree == max(degrees.values())]
            
        #     if len(max_degree_nodes) == 0 or subgraph.degree(max_degree_nodes[0]) == 0:
        #         break  # No more nodes to remove or all remaining nodes have degree 0

        #     subgraph.remove_node(max_degree_nodes[0])
        # nodes = list(subgraph)
        # nodes.sort()
        # print(nodes)
        # subgraph = datasets[dataset_index[solution["dataset_name"]]]["graph"].subgraph(
        #     IS_set
        # )
        # if len(subgraph.edges) > 0:
        #     plt.figure(i)
        #     plt.title(subgraph.edges)
        #     i += 1
        #     is_check.append(False)
        #     print(
        #         f"Non IS found using {solution['solution_method']} on {solution['dataset_name']}"
        #     )
        is_check.append(True)


    table_row = [dataset_solutions[0]['dataset_name']]

    table_row.extend([solution["data"]["size"] for solution in dataset_solutions])
    table_row.extend([solution["time_taken"] for solution in dataset_solutions])
    table_row.extend(is_check)

    table_data.append(table_row)

table_headers = ["Dataset Name"]

table_headers.extend([solution["solution_method"] + " Solution Size" for solution in dataset_solutions])
table_headers.extend([solution["solution_method"] + " Solution Time" for solution in dataset_solutions])
table_headers.extend([solution["solution_method"] + " Solution IS" for solution in dataset_solutions])

table = pandas.DataFrame(table_data, columns=table_headers)
table
table.to_csv("results.csv")

IndexError: list index out of range

## Optional: Save Solutions to Pickle

In [ ]:
import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

def read_object(filename):
    with open(filename, 'rb') as outp:  # Overwrites any existing file.
        return pickle.load(outp)

# solutions = save_object(solutions, "solutions.pkl")